# 算法 8.1（AdaBoost）
- 测试数据：例8.1，表8.1训练数据表

In [ ]:
import numpy as np
import math

class BaseClassifier(object):
    def __init__(self):
        self._error = None

    def fit(self, x_data, y_data):
        pass
    def predict(self, x_test):
        pass
    @property
    def error(self):
        return self._error

class Adaboost(object):
    def __init__(self, classifier = None, classifier_count = 10):
        if classifier is None:
            self._classifier = BaseClassifier()
        self._x_train = None
        self._y_train = None
        self._weight = None
        self._error = None
        self._alpha = None

        self._classifier_count = classifier_count
        self._weights = []
        self._errors = []
        self._alphas = []
        self._funcs = []

    def fit(self, x_train, y_train):
        self._x_train = x_train
        self._y_train = y_train
        self._m = len(self._x_train)
        self._n = len(self._x_train[0])
        # 初始化权值
        self._weight = [1. / self._m]*self._m
        self._weights.append(self._weight)
        x = self._weight * self._x_train
        for i in range(self._classifier_count):
            self._classifier.fit(x, self._y_train)
            alpha = 1. / 2 * math.log((1 - self._classifier.error) / self._classifier.error)
            self._weight = self._get_weight()
            self._alphas.append(alpha)


    def predict(self, x_test):
        pass

    @property
    def x_train(self):
        return self._x_train
    @property
    def y_train(self):
        return self._y_train
    
    def _get_weight(self):
        pass

x_train = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]).reshape(-1, 1)
y_train = np.array([1, 1, 1,-1,-1,-1, 1, 1, 1,-1]).reshape(-1, 1)

ada = Adaboost()
ada.fit(x_train, y_train)
